In [27]:
%reload_ext autoreload
%autoreload 2

from npfd.models.base import HiddenMarkovModel
from npfd import data

from npfd import visualization as viz


In [2]:
from verta import Client

HOST = "http://127.0.0.1:3000/"

PROJECT_NAME = "NPF Detector"
EXPERIMENT_NAME = "Test_notebook"
EXPERIMENT_DESCRIPTION = "Data Version -> 2. Adapted model to real labels.Notebook -> 2-EventNoEvent_HInit_normalized_adapted.ipynb"


client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)
run = client.set_experiment_run(desc=EXPERIMENT_DESCRIPTION)

ModuleNotFoundError: No module named 'verta'

In [28]:
SEARCH_PARAMS = {  'normalize': True,
                   'data_version': '2',
                   
                   # Labels
                   'label_type': 'event-noevent',
                   'nuc_threshold': 0.15,        # 1/cm3/10min
                   'pos_vol_threshold': 200,     # 1/m3^3/10min
                   'neg_vol_threshold': -5000,    # 1/cm3/10min
                   
                   # Initialization
                   'variance_floor': 0.00014,
                   'minimum_variance': 1.6e-7,
                 
                   'word_insertion_penalty': 7.25,
                   'grammar_scale_factor': 5.1
                  }  

params = {'init_metho': 'HCompV',
          'raw_data_source': 'simulation',
          **SEARCH_PARAMS}

# run.log_hyperparameters(params)

In [29]:
# Make datasets
params['raw_data_source'] = 'malte-uhma'
X_train, X_val, y_train, y_val = data.dataset.make_dataset(params, clean_interim_dir=False)

params['raw_data_source'] = 'real'
X_adapt, X_final, y_adapt, y_final = data.dataset.make_dataset(params, test_size=0.9)


# run.log_observation('number_of_test_files', X_val['count'])
# run.log_observation('number_of_train_files', X_train['count'])#

INFO:root:Converting malte-uhma raw files to HTK format ...
INFO:root:Generating script (.scp) files...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...
INFO:root:Adding deltas and acelerations...
INFO:root:Test files:	1812
INFO:root:Train files:	5058
INFO:root:Converting real raw files to HTK format ...
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8921: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cond, other, inplace, axis, level, errors=errors, try_cast=try_cast
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...


In [24]:
# Initialize model
# run = client.set_experiment_run(desc=EXPERIMENT_DESCRIPTION)

model = HiddenMarkovModel()

model.initialize(X_train, params)


model.train(X_train, y_train, params)
results = model.test(X_val, y_val, params)

# for k,v in results.items():
#     run.log_observation(k,v)
    
    
edit_commands = ['MU 2 {*.state[2-4].mix}']

model.edit(edit_commands)
model.train(X_train, y_train, params)

edit_commands = ['MU 4 {*.state[2-4].mix}']

model.edit(edit_commands)
model.train(X_train, y_train, params)

edit_commands = ['MU 8 {*.state[2-4].mix}']

model.edit(edit_commands)
model.train(X_train, y_train, params)
    
results = model.test(X_val, y_val, params)

# for k,v in results.items():
#     run.log_observation(k,v)

# for command in edit_commands:    
#     run.log_observation('model_edit_commands', command)
    
# Adapt the model    
print('Adapting...')
model.adapt(X_adapt, y_adapt)

results = model.test(X_final, y_final, params)

# for k,v in results.items():
#     run.log_observation(k,v)

# Log final results    
# run.log_metric('correct_labels', results['WORD_Corr'])
# run.log_artifact('final_model', '../models/hmm/' + str(model.most_trained_model), overwrite=True)

INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...


Calculating Fixed Variance
  HMM Prototype: /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/proto
  Segment Label: None
  Num Streams  : 1
  UpdatingMeans: Yes
  Target Direct: /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/0
241776 speech frames accumulated
Updating HMM Means and Covariances
Output written to directory /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/0
Var floor macros output to file /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/0/vFloors

/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks
Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest

Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest



INFO:root:Most trained model: 3
INFO:root:Testing model: 3


Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest



INFO:root:Editing model 3
INFO:root:Most trained model: 4
INFO:root:Training the model...


...

Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest

HHEd
 3/3 Models Loaded [5 states max, 4 mixes max]

MU 8 {}
 Mixup to 8 components per stream
 MU: Number of mixes increased from 36 to 72

Saving new HMM files ...
Edit Complete

Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest

Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest



INFO:root:Most trained model: 15
INFO:root:Testing model: 15


Pruning-Off
 WARNING [-2331]  UpdateModels: na[2] copied: only 0 egs
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest



INFO:root:Adapting model 15
INFO:root:Testing model: 15
INFO:root:Testing real data


/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/15/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/15/hmmdefs -p 7.2500000000 -s 5.1000000000 -A -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/data/../../data/interim/test.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/wdnet /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Sat Jun 19 23:55:14 2021
  Ref : >acultad/Tesis/models/bdb/npfd/data/../../data/interim/test_labels.mlf
  Rec : >gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/res

In [214]:
viz.visualize.generate_plots('real_data', X_final, y_final, results)

IndexError: list index out of range

In [132]:
viz.visualize.generate_plots('real_data_for_adaptation', X_adapt, y_adapt, results)

Error in label of date: 20170116
Error in label of date: 20171009


In [136]:
final_results = model.predict(X_final, params)
viz.visualize.generate_plots('final_resultos', X_final, final_results)

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/./HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/../../models/hmm/7/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/../../models/hmm/7/hmmdefs -A -p 10.0000000000 -s 10.0000000000 -J /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/../../models/classes/ -J /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/../../models/xforms/ mllr1 -h */%* -k -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/./HTK/misc/wdnet -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/data/../../data/interim/2015.test_D_A.real.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/../../data/interim/results.mlf /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/./HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/./HTK/